# Canadian Personal Income Statistics

The graphs below show personal income statistics for Canadians using data from [Statistics Canada](https://doi.org/10.25318/1110000801-eng).  

Total Income includes income from:
- employment income (salaries, commission), 
- self employment income, pension income (OAS, CPP/QPP, registered pension plans, RRIFs), 
- investment income, 
- social benefit payments (EI, workers' compensation, social assisstance), and 
- other income.

In [1]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import ipywidgets as widgets
import logging
import sys # for relative imports
sys.path.append('..')

# local imports
from wrangling import subset_plot_data_for_income_bins
from wrangling import (
    preprocess_income_bin_data,
    subset_year_age_sex_geo
)
from visualize import set_chart_title, create_bar_chart
from load_data import load_csv_table

logger = logging.getLogger(__name__)
FORMAT = "[%(filename)s:%(lineno)s - %(funcName)20s() ] %(message)s"
logging.basicConfig(format=FORMAT)
logger.setLevel(logging.DEBUG)

In [2]:
#load the data
df = pd.read_csv(r"../data/processed/11100008.csv")

In [3]:
# parameters
age_values = ['All age groups', '0 to 24 years', '25 to 34 years',
       '35 to 44 years', '45 to 54 years', '55 to 64 years',
       '65 to 74 years', '75 years and over', '65 years and over']
geo_values = ['Canada', 'Newfoundland and Labrador',
       "St. John's, Newfoundland and Labrador", 'Prince Edward Island',
       'Nova Scotia', 'Halifax, Nova Scotia', 'New Brunswick',
       'Saint John, New Brunswick', 'Quebec', 'Saguenay, Quebec',
       'Québec, Quebec', 'Sherbrooke, Quebec', 'Trois-Rivières, Quebec',
       'Montréal, Quebec', 'Ottawa-Gatineau, Quebec part', 'Ontario',
       'Ottawa-Gatineau, Ontario part', 'Oshawa, Ontario',
       'Toronto, Ontario', 'Hamilton, Ontario',
       'St. Catharines-Niagara, Ontario',
       'Kitchener-Cambridge-Waterloo, Ontario', 'London, Ontario',
       'Windsor, Ontario', 'Greater Sudbury, Ontario',
       'Thunder Bay, Ontario', 'Manitoba', 'Winnipeg, Manitoba',
       'Saskatchewan', 'Regina, Saskatchewan', 'Saskatoon, Saskatchewan',
       'Alberta', 'Calgary, Alberta', 'Edmonton, Alberta',
       'British Columbia', 'Vancouver, British Columbia',
       'Victoria, British Columbia', 'Yukon', 'Northwest Territories',
       'Nunavut', 'Kingston, Ontario',
       'Abbotsford-Mission, British Columbia', 'Moncton, New Brunswick',
       'Peterborough, Ontario', 'Brantford, Ontario', 'Guelph, Ontario',
       'Barrie, Ontario', 'Kelowna, British Columbia',
       'Bay Roberts, Newfoundland and Labrador',
       'Grand Falls-Windsor, Newfoundland and Labrador',
       'Corner Brook, Newfoundland and Labrador',
       'Non CMA-CA, Newfoundland and Labrador',
       'Charlottetown, Prince Edward Island',
       'Summerside, Prince Edward Island',
       'Non CMA-CA, Prince Edward Island', 'Kentville, Nova Scotia',
       'Truro, Nova Scotia', 'New Glasgow, Nova Scotia',
       'Cape Breton, Nova Scotia', 'Non CMA-CA, Nova Scotia',
       'Fredericton, New Brunswick', 'Bathurst, New Brunswick',
       'Miramichi, New Brunswick', 'Campbellton, New Brunswick part',
       'Edmundston, New Brunswick', 'Non CMA-CA, New Brunswick',
       'Campbellton, Quebec part', 'Matane, Quebec', 'Rimouski, Quebec',
       'Rivière-du-loup, Quebec', 'Baie-Comeau, Quebec', 'Alma, Quebec',
       'Dolbeau-Mistassini, Quebec', 'Sept-Îles, Quebec',
       'Saint-Georges, Quebec', 'Thetford Mines, Quebec',
       'Cowansville, Quebec', 'Victoriaville, Quebec',
       'Shawinigan, Quebec', 'La Tuque, Quebec', 'Drummondville, Quebec',
       'Granby, Quebec', 'Saint-Hyacinthe, Quebec', 'Sorel-Tracy, Quebec',
       'Joliette, Quebec', 'Saint-Jean-sur-Richelieu, Quebec',
       'Salaberry-de-Valleyfield, Quebec', 'Lachute, Quebec',
       "Val-d'Or, Quebec", 'Amos, Quebec', 'Rouyn-Noranda, Quebec',
       'Hawkesbury, Quebec part', 'Non CMA-CA, Quebec',
       'Cornwall, Ontario', 'Hawkesbury, Ontario part',
       'Brockville, Ontario', 'Pembroke, Ontario', 'Petawawa, Ontario',
       'Belleville, Ontario', 'Cobourg, Ontario', 'Port Hope, Ontario',
       'Kawartha Lakes, Ontario', 'Centre Wellington, Ontario',
       'Ingersoll, Ontario', 'Woodstock, Ontario', 'Tillsonburg, Ontario',
       'Norfolk, Ontario', 'Stratford, Ontario', 'Chatham-Kent, Ontario',
       'Leamington, Ontario', 'Sarnia, Ontario', 'Owen Sound, Ontario',
       'Collingwood , Ontario', 'Orillia, Ontario', 'Midland, Ontario',
       'North Bay, Ontario', 'Elliot Lake, Ontario',
       'Temiskaming Shores, Ontario', 'Timmins, Ontario',
       'Sault Ste. Marie, Ontario', 'Kenora, Ontario',
       'Non CMA-CA, Ontario', 'Portage la Prairie, Manitoba',
       'Brandon, Manitoba', 'Thompson, Manitoba', 'Non CMA-CA, Manitoba',
       'Yorkton, Saskatchewan', 'Moose Jaw, Saskatchewan',
       'Swift Current, Saskatchewan', 'North Battleford, Saskatchewan',
       'Prince Albert, Saskatchewan', 'Estevan, Saskatchewan',
       'Lloydminster, Saskatchewan part', 'Non CMA-CA, Saskatchewan',
       'Medicine Hat, Alberta', 'Brooks, Alberta', 'Lethbridge, Alberta',
       'Okotoks, Alberta', 'Canmore, Alberta', 'Red Deer, Alberta',
       'Camrose, Alberta', 'Lloydminster, Alberta part',
       'Cold Lake, Alberta', 'Grande Prairie, Alberta',
       'Wood Buffalo, Alberta', 'Wetaskiwin, Alberta',
       'Non CMA-CA, Alberta', 'Cranbrook, British Columbia',
       'Penticton, British Columbia', 'Vernon, British Columbia',
       'Salmon Arm, British Columbia', 'Kamloops, British Columbia',
       'Chilliwack, British Columbia', 'Squamish, British Columbia',
       'Duncan, British Columbia', 'Nanaimo, British Columbia',
       'Parksville, British Columbia', 'Port Alberni, British Columbia',
       'Courtenay, British Columbia', 'Campbell River, British Columbia',
       'Powell River, British Columbia',
       'Williams Lake, British Columbia', 'Quesnel, British Columbia',
       'Prince Rupert, British Columbia', 'Kitimat, British Columbia',
       'Terrace, British Columbia', 'Prince George, British Columbia',
       'Dawson Creek, British Columbia',
       'Fort St. John, British Columbia', 'Non CMA-CA, British Columbia',
       'Whitehorse, Yukon', 'Non CMA-CA, Yukon',
       'Yellowknife, Northwest Territories',
       'Non CMA-CA, Northwest Territories',
       'Campbellton, combined NewBrunswick/Quebec',
       'Hawkesbury, combined Ontario/Quebec',
       'Ottawa - Gatineau, combined Ontario/Quebec',
       'Lloydminster, combined Alberta/Saskatchewan',
       'Steinbach, Manitoba', 'High River, Alberta',
       'Strathmore, Alberta', 'Sylvan Lake, Alberta', 'Lacombe, Alberta',
       'Gander, Newfoundland and Labrador', 'Sainte-Marie, Quebec',
       'Arnprior, Ontario', 'Carleton Place, Ontario',
       'Wasaga Beach, Ontario', 'Winkler, Manitoba',
       'Weyburn, Saskatchewan', 'Nelson, British Columbia']
sex_values = ['Both sexes', 'Males', 'Females']
year_values = [2000, 2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010,
       2011, 2012, 2013, 2014, 2015, 2016, 2017]

In [5]:
initial_age = "35 to 44 years"
initial_year = 2017
initial_geo = "Canada"
initial_sex = "Females"



select_age = widgets.Dropdown(options=age_values, value=initial_age)
select_year = widgets.Dropdown(options=year_values, value=initial_year)
select_geo = widgets.Dropdown(options=geo_values, value=initial_geo)
select_sex = widgets.Dropdown(options=sex_values, value=initial_sex)


df = subset_year_age_sex_geo(df, initial_year, initial_age, initial_sex, initial_geo)
hist, cumulative_plot = create_bar_chart(df, initial_year, initial_age, initial_sex, initial_geo)
hist = go.FigureWidget(hist)
cumulative_plot = go.FigureWidget(cumulative_plot)

def update_bar_charts_callback(ignore):
    global hist
    global cumulative_plot
    global df
    year = select_year.value
    geo = select_geo.value
    age = select_age.value
    sex = select_sex.value
    
    df = subset_year_age_sex_geo(df, year, age, sex, geo)
    df_subset = subset_plot_data_for_income_bins(df)
    y_hist, y_cumulative = preprocess_income_bin_data(df_subset)
    hist.data[0]['y'] = y_hist
    hist.update_layout(title=set_chart_title("hist", year, geo, sex, age))
    cumulative_plot.data[0]['y'] = y_cumulative
    cumulative_plot.update_layout(title=set_chart_title("cumulative", year, geo, sex, age))
    
select_age.observe(update_bar_charts_callback, "value")
select_geo.observe(update_bar_charts_callback, "value")
select_year.observe(update_bar_charts_callback, "value")
select_sex.observe(update_bar_charts_callback, "value")


bar_chart_dropdowns = widgets.HBox([select_year, select_geo, select_age, select_sex])

In [6]:
bar_charts = widgets.HBox([hist, cumulative_plot])
bar_charts.layout.flex_flow = "row wrap"
bar_charts.layout.justify_content="center"
bar_charts.layout.flex_grow='1'

In [7]:
layout = widgets.VBox([bar_chart_dropdowns, bar_charts])
layout